In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
data_dir="G:\Banglalink\Python Master\PythonForModellers-master\Data"
os.chdir(data_dir)

In [ ]:
data=pd.read_csv("dm.csv",na_values=[""," ","NA","N/A"])

In [ ]:
data.head()

In [ ]:
## Assume people who spend more than the average are good customers
data['target']=data['AmountSpent'].map(lambda x: 1 if x>data['AmountSpent'].mean() else 0)

In [ ]:
data=data.drop("AmountSpent",axis=1)

In [ ]:
data.head()

In [ ]:
data['History'].value_counts()

In [ ]:
data['History'].isnull().sum()

In [ ]:
## Minimal Data Prep
data['History']=data['History'].fillna("NewCust")

In [ ]:
data.head()

In [ ]:
## Split the data into test and train
data_train=data.sample(frac=0.70,random_state=200)
data_test=data.drop(data_train.index)

In [ ]:
## Build Model
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [ ]:
model1=smf.glm("target~C(Age)+C(Gender)+C(OwnHome)+C(Married)+C(Location)+Salary+Children+C(History)+Catalogs",data=data_train,
              family=sm.families.Binomial()).fit()

In [ ]:
print(model1.summary())

In [ ]:
## Variables to exclude
#Age
#Gender
#Ownhome
#Married
## Variables for dummy creation
#Hist_Low
#Hist_Medium

data_train['Hist_Low']=data_train['History'].map(lambda x: 1 if x=="Low" else 0)
data_test['Hist_Low']=data_test['History'].map(lambda x: 1 if x=="Low" else 0)
data_train['Hist_Med']=data_train['History'].map(lambda x: 1 if x=="Medium" else 0)
data_test['Hist_Med']=data_test['History'].map(lambda x: 1 if x=="Medium" else 0)

In [ ]:
model2=smf.glm("target~Children+Catalogs+Salary+Hist_Med",data=data_train,
              family=sm.families.Binomial()).fit()

In [ ]:
print(model2.summary())

In [ ]:
## Let's check confusion matrix and AUC
import sklearn.metrics as metrics

In [ ]:
y_true=data_test['target']
y_pred=model2.predict(data_test)

In [ ]:
y_pred.head()

In [ ]:
y_true=data_test['target']
y_pred=model2.predict(data_test).map(lambda x:1 if x>0.5 else 0)
metrics.confusion_matrix(y_true,y_pred)

In [ ]:
## ROC curve
y_score=model2.predict(data_test)
fpr,tpr,thresholds=metrics.roc_curve(y_true,y_score)
x,y=np.arange(0,1.1,0.1),np.arange(0,1.1,0.1)

In [ ]:
plt.plot(fpr,tpr,"-")
plt.plot(x,y,'b--')

In [ ]:
## AUC
metrics.roc_auc_score(y_true,y_score)

In [ ]:
## Gains
data_test['prob']=model2.predict(data_test)

In [ ]:
data_test['prob'].head()

In [ ]:
data_test['prob_deciles']=pd.qcut(data_test['prob'],q=10)

In [ ]:
data_test.head()

In [ ]:
data_test.sort_values('prob',ascending=False).head()

In [ ]:
gains=data_test.groupby("prob_deciles",as_index=False)['target'].agg(['sum','count']).reset_index().sort_values("prob_deciles",
                 ascending=False)

In [ ]:
gains.columns=["Deciles","TotalEvents","NumberObs"]

In [ ]:
gains["PercEvents"]=gains['TotalEvents']/gains['TotalEvents'].sum()

In [ ]:
gains["CumulativeEvents"]=gains.PercEvents.cumsum()

In [ ]:
gains

In [ ]:
data_test.sort_values("prob",ascending=False)[['Cust_Id']].head(90)## These are the people to target